In [1]:
import numpy as np
import math
import pandas as pd
from collections import Counter
import glob
import os
from copy import deepcopy
import re

mutation_rate = pd.read_table("../../Cache/Subchallenge2/mutation_prob2.tsv")
mutation_rate = mutation_rate[mutation_rate["is_leaf"]==1]
mutation_rate = mutation_rate[mutation_rate["prob"]!=0]
mutation_rate = dict(zip(mutation_rate["state"],mutation_rate["prob"]))
mutation_rate.pop("-")
mutation_rate["0"] = 1 - sum(mutation_rate.values())
mutation_rate["-"] = mutation_rate["0"]
mutation_rate

{'A': 0.12536658446711108,
 'B': 0.06486985138554949,
 'C': 0.0393033142917129,
 'D': 0.025985296046771487,
 'E': 0.01705983510240882,
 'F': 0.012359821058990441,
 'G': 0.008742719110672443,
 'H': 0.0056820632444791715,
 'I': 0.003641818346727136,
 'J': 0.0022862019242180357,
 'K': 0.0014439455698432811,
 'L': 0.0009523116719154326,
 'M': 0.0005861017898258615,
 'N': 0.00035103436909203635,
 'O': 0.000248648942793604,
 'P': 0.00014147152770721048,
 'Q': 6.943615043928041e-05,
 'R': 4.17128864653149e-05,
 'S': 2.7688571280571998e-05,
 'T': 1.4063186923999163e-05,
 'U': 9.115649855956533e-06,
 'V': 1.227178618482966e-06,
 'W': 3.5976471269906852e-06,
 '0': 0.690812139879471,
 '-': 0.690812139879471}

In [2]:
class TreeNode(object):
    def __init__(self,label=None,seq=None,parent=None,leaves=[]):
        self.label = label
        self.seq = seq
        self.left = None
        self.right = None
        self.parent = parent
        self.leaves = leaves

In [3]:
class hcluster(object):
    def __init__(self,mutation_rate,df):
        self.mutation_rate = mutation_rate
        self.df = df
        self.all_tree_nodes = [TreeNode(label=i[1].label,seq=i[1].seq,\
                                       leaves=[i[0]]) for i in df.iterrows()]
        self.root = TreeNode(leaves=[i for i in range(len(self.all_tree_nodes))])
    
    def initialize_dist_matrix(self):
        n = len(self.all_tree_nodes)
        m = len(self.all_tree_nodes[0].seq)
        
        D = np.zeros((n,n))
        D2 = np.zeros((n,n))
        for i in range(n):
            for j in range(i):
                D[i,j] = self.sequence_dist(self.all_tree_nodes[i].seq,self.all_tree_nodes[j].seq)
                D2[i,j] = self.sequence_dist2(self.all_tree_nodes[i].seq,self.all_tree_nodes[j].seq)
        D2 = D2 + D2.T
        D = D + D.T
        
        return D + self.lambda1 * D2 + self.lambda2 * self.get_deletion_matrix()
    
    def get_deletion_matrix(self):
        n = len(self.df)
        m = len(self.df["seq"][0])
        deletions = pd.DataFrame(data={"label":[],"start":[],"end":[]})
        p = re.compile("-+")

        for i in range(n):
            s = self.df["seq"][i]
            for j in p.finditer(s):
                deletions = deletions.append({"label":self.df["label"][i],"start":j.start(),"end":j.end()},ignore_index=True)
        deletion_summary = deletions.groupby(["start","end"]).size().reset_index()
        deletion_summary.columns = ["start","end","count"]
        deletion_summary = deletion_summary[deletion_summary["count"]>=2].sort_values("count")

        labels = self.df["label"].tolist()
        clusters = []
        for i,row in deletion_summary.iterrows():
            start = row["start"]
            end = row["end"]
            cl_labels = deletions[(deletions["start"]==start) & (deletions["end"]==end)]
            cl_labels = cl_labels["label"].tolist()
            clusters.append([labels.index(j) for j in cl_labels])

        deletion_matrix = np.zeros((n,n))
        for c in clusters:
            deletion_matrix[np.ix_(c,c)] = 1

        np.fill_diagonal(deletion_matrix,0)
        return deletion_matrix
    
    def sequence_dist(self,s1,s2):
        def match_prob(a,b):
            if a != "0" and a != "-" and a == b:
                return math.log(self.mutation_rate[a])
            return 0
        return sum([match_prob(s1[i],s2[i]) for i in range(len(s1))])
  
    def sequence_dist2(self,s1,s2):
        def mismatch_prob(a,b):
#             if a == "-" or b == "-":
#                 return -math.log(self.mutation_rate["0"]*self.mutation_rate["0"])
            if a != "0" and b != "0" and a == b:
                return 0   
            return -math.log(self.mutation_rate[a]*self.mutation_rate[b])
        
        return sum([mismatch_prob(s1[i],s2[i]) for i in range(len(s1))])

    
    def merge_node(self,D):
        np.fill_diagonal(D,float("inf"))
        i = np.argmin(D)
        i,j = i//len(D),i%len(D)
        return i,j
    
    def update_dist_matrix(self,clusters):
        def cluster_dist(c1,c2):
            d = 0
            for cc1 in c1:
                for cc2 in c2:
                    d += self.D[cc1,cc2]
            return d/(len(c1)*len(c2))
        
        n = len(clusters)
        D = np.zeros((n,n))
        
        for i in range(n):
            for j in range(i):
                D[i,j] = cluster_dist(clusters[i].leaves,clusters[j].leaves)
                
        D = D + D.T
        return D
    
    def h_clustering(self,lambda1=0.1,lambda2=-10):
        self.lambda1 = lambda1
        self.lambda2 = lambda2
        clusters = self.all_tree_nodes
#         print([i.leaves for i in clusters])
        self.D = self.initialize_dist_matrix()
        
        D = self.D
        nn = len(self.D)
        
        for c in range(nn-1):
#             print("Dist matrix")
#             print(np.around(D,decimals=1))
            n = len(D)
            i,j = self.merge_node(D)
            parent = TreeNode()
            parent.left = clusters[i]
            parent.right = clusters[j]
#             print("joining nodes: %s and %s" \
#               %(' '.join([str(c) for c in clusters[i].leaves]),\
#                 ' '.join([str(c) for c in clusters[j].leaves])))
            parent.leaves = clusters[i].leaves + clusters[j].leaves
            clusters = [clusters[idx] for idx in range(len(clusters)) if idx!=i and idx!=j] + [parent]
#             print([i.leaves for i in clusters])
            D = self.update_dist_matrix(clusters)
        self.root = parent
    
    def write_newick(self):
        def helper(root):
            if not root:
                return

            if not root.left and not root.right:
                return "%s" % (root.label)

            return "(%s,%s)" % (helper(root.left),helper(root.right))

        return helper(self.root) + "root;"

In [4]:
# all_trees = [os.path.basename(f) for f in glob.glob("../../Cache/Subchallenge2/10_leaves/gt/*.nw")]
# for tree_name in all_trees:
#     df = pd.read_csv("../../Cache/Subchallenge2/10_leaves/df/%s.csv" % tree_name.split(".")[0])
#     t = hcluster(mutation_rate,df)
#     t.h_clustering(lambda1=0.1,lambda2=-10)
#     outfile = "../../Cache/Subchallenge2/10_leaves/hc2/%s" % tree_name
#     f = open(outfile,"w")
#     f.write(t.write_newick())
#     f.close()

In [5]:
# 0.1 -10

# def solver(sample):
#     df = pd.read_table("../../Data/Subchallenge2/SubC2_train_TXT/%s.txt"%sample,\
#                        header=None,names=["label","seq"])
#     t = hcluster(mutation_rate,df)
#     t.h_clustering(lambda1=0.1,lambda2=-10)
#     outfile = "../../Output/Subchallenge2/Customized_hcluster2/%s.nw" % sample
#     f = open(outfile,"w")
#     f.write(t.write_newick())
#     f.close()

In [6]:
# from multiprocessing import Process, Pool

# samples = [i.split(".")[0] for i in os.listdir("../../Data/Subchallenge2/SubC2_train_TXT/")]
# num_threads = 30
# pool = Pool(num_threads)
# pool.map(solver,samples)

In [7]:
test_df = pd.read_table("../../Data/Subchallenge2/SubC2_TEST_data.txt",\
                       sep=" ",header=None,names=["label","seq"])
t = hcluster(mutation_rate,test_df)
t.h_clustering(lambda1=0.1,lambda2=-10)
outfile = "../../Output/Subchallenge2/Jasper_Hu_sub2_submission.nw"
f = open(outfile,"w")
f.write(t.write_newick())
f.close()